# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
491,YUM
492,ZBH
493,ZBRA
494,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from my_secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}
#https://cloud.iexapis.com/stable/tops?token={IEX_CLOUD_API_TOKEN}&symbols={symbol}
data = requests.get(api_url).json()
price = data['iexClose']
print(data)
price

{'avgTotalVolume': 62011003, 'calculationPrice': 'tops', 'change': 1.1, 'changePercent': 0.00579, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 191.09, 'iexAskSize': 100, 'iexBidPrice': 191.07, 'iexBidSize': 200, 'iexClose': 191.08, 'iexCloseTime': 1716916653314, 'iexLastUpdated': 1716916653314, 'iexMarketPercent': 0.017656381543431198, 'iexOpen': 191.56, 'iexOpenTime': 1716903000006, 'iexRealtimePrice': 191.08, 'iexRealtimeSize': 3, 'iexVolume': 481699, 'lastTradeTime': 1716916648718, 'latestPrice': 191.08, 'latestSource': 'IEX real time price', 'latestTime': '1:17:33 PM', 'latestUpdate': 1716916653314, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2930036388560, 'oddLot

191.08

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:

# URL for getting stock stats including marketCap
stats_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
stats_response = requests.get(stats_url, verify=True)
stats_data = stats_response.json()
print(stats_data)
marketCap = stats_data.get('marketcap', 'N/A')
marketCap

{'companyName': 'Apple Inc', 'marketcap': 2913168898360, 'week52high': 198.57, 'week52low': 163.63, 'week52highSplitAdjustOnly': 199.62, 'week52lowSplitAdjustOnly': 164.08, 'week52change': 0.09447084253169669, 'sharesOutstanding': 15334082000, 'float': 0, 'avg10Volume': 41730424, 'avg30Volume': 62011003, 'day200MovingAvg': 182.3, 'day50MovingAvg': 177.64, 'employees': 164000, 'ttmEPS': 6.43, 'ttmDividendRate': 0.966204324874062, 'dividendYield': 0.005085821269997168, 'nextDividendDate': '', 'exDividendDate': '2024-05-10', 'nextEarningsDate': '2024-08-01', 'peRatio': 29.01880582892548, 'beta': 1.0670495890183638, 'maxChangePercent': 73.6483300589391, 'year5ChangePercent': 3.4285724276027016, 'year2ChangePercent': 0.2923897441479193, 'year1ChangePercent': 0.09447084253169669, 'ytdChangePercent': -0.008040417732047778, 'month6ChangePercent': 0.006280913290768408, 'month3ChangePercent': 0.043068437434286455, 'month1ChangePercent': 0.12519530116357425, 'day30ChangePercent': 0.12519530116357

2913168898360

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:

my_columns = ['Ticker','Stock Price','Market Capitalization','Number Of Shares To Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy


In [7]:
# Create the Series to append
new_series = pd.Series(
    [symbol, price, marketCap, 'N/A'],
    index=my_columns
)

# Use pd.concat to append the new series to the DataFrame
final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)

# Print the updated DataFrame
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,AAPL,191.08,2913168898360,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][400:]:
    try:
        # Fetch stats data
        stats_url = f'https://cloud.iexapis.com/stable/stock/{stock}/stats?token={IEX_CLOUD_API_TOKEN}'
        stats_response = requests.get(stats_url, verify=True)
        
        if stats_response.status_code != 200:
            print(f"Error fetching stats for {stock}: {stats_response.status_code}")
            print(stats_response.text)
            continue
        
        stats_data = stats_response.json()
        
        # Fetch quote data
        api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
        data_response = requests.get(api_url, verify=True)
        
        if data_response.status_code != 200:
            print(f"Error fetching quote for {stock}: {data_response.status_code}")
            print(data_response.text)
            continue
        
        data = data_response.json()
        
        # Create new series for the DataFrame
        new_series = pd.Series(
            [stock, data.get('iexClose', 'N/A'), stats_data.get('marketcap', 'N/A'), 'N/A'],
            index=my_columns
        )
        
        # Append the new series to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
    
    except requests.exceptions.RequestException as e:
        print(f"RequestException for {stock}: {e}")
    except requests.exceptions.JSONDecodeError as e:
        print(f"JSONDecodeError for {stock}: {e}")
        print("Response content:", data_response.text)

# Print the updated DataFrame
print(final_dataframe)

   Ticker Stock Price Market Capitalization Number Of Shares To Buy
0    SCHW       71.09          128444119840                     N/A
1     SEE       36.81            5421234090                     N/A
2     SHW      303.55           77291892097                     N/A
3    SIVB       39.05                     0                     N/A
4     SJM       109.1           11654890124                     N/A
..    ...         ...                   ...                     ...
91    YUM      136.83           38766673982                     N/A
92    ZBH      117.11           23948842811                     N/A
93   ZBRA      326.53           16814144781                     N/A
94   ZION       42.87            6362442402                     N/A
95    ZTS      170.67           77602113950                     N/A

[96 rows x 4 columns]


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [8]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [9]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [symbol, data[symbol]['quote'].get('iexClose', 'N/A'), data[symbol]['stats'].get('marketcap', 'N/A'), 'N/A'],
            index=my_columns
        )

        # Use pd.concat to append the new series to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,A,148.65,44151709087,N/A
1,AAL,13.655,9080290361,N/A
2,AAP,71.1,4147030897,N/A
3,AAPL,191.09,2913168898360,N/A
4,ABBV,154.75,277347193684,N/A
...,...,...,...,...
491,YUM,136.41,38766673982,N/A
492,ZBH,116.41,23948842811,N/A
493,ZBRA,325.66,16814144781,N/A
494,ZION,42.685,6362442402,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [10]:

portfolio_size = input('Enter your portfolio size: ')
val = float(portfolio_size)
print(val)


10000000.0


In [11]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number Of Shares To Buy']=math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,A,148.65,44151709087,135
1,AAL,13.655,9080290361,1476
2,AAP,71.1,4147030897,283
3,AAPL,191.09,2913168898360,105
4,ABBV,154.75,277347193684,130
...,...,...,...,...
491,YUM,136.41,38766673982,147
492,ZBH,116.41,23948842811,173
493,ZBRA,325.66,16814144781,61
494,ZION,42.685,6362442402,472


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object
### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [12]:


# Define the Excel writer and the sheet name
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

# Write the DataFrame to an Excel sheet
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)

# Get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Recommended Trades']

# Define formats
background_color = '#0a0a24'
font_color = '#ffffff'

string_format = workbook.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = workbook.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = workbook.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

# Apply formats to columns
for col_num, col_name in enumerate(my_columns):
    if col_name == 'Stock Price':
        worksheet.set_column(col_num, col_num, 18, dollar_format)
    elif col_name == 'Market Capitalization':
        worksheet.set_column(col_num, col_num, 18, dollar_format)
    else:
        worksheet.set_column(col_num, col_num, 18, string_format)

# Close the Excel writer
writer.close()


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops: